# Using Azure Monitor Logs extension in Azure Data Studio

## Triangle SQL Server User Group

## 

October 17, 2023

Written by:  Julie Koesmarno

Original Notebook: [Using Azure Monitor Logs extension in Azure Data Studio Insiders](https:\github.com\MsSQLGirl\jubilant-data-wizards\blob\main\Simple%2520Demo\KQL%2520Notebooks\AzureMonitorLogsSample.ipynb)

Modifed by: Taiob  Ali

SqlWorldWide.com

Once Azure Monitor Logs (Preview) extension installed in Azure Data Studio Insiders build, you can connect to your Azure Monitor Log workspace(s), browse the tables, write/execute KQL queries against workspaces and write/execute Notebooks connected to the Azure Monitor Log kernel.

There are two main tables in Azure Log Analytics (Azure Monitor Logs) workspace that capture Azure SQL events:

1. AzureDiagnostics
2. AzureMetric

## 1\. Connect to Azure Monitor Logs (Log Analytics) workspace

Workspace is similar to what a database is to SQL. You connect to Log Analytics workspace to start querying data.

### 1.1 Install Azure Monitor Logs extension first

Go to the Extension viewlet and type in Azure Monitor Logs. Install it and restart ADS.

### 1.2 Connect to the desired Azure Monitor Logs workspace

Change the Kernel to "Log Analytics". Set Attach to to a new or existing connection to the workspace. Note: you will need a workspace Id that you can obtain from Azure portal.

> **Note**: the name of the kernel is subject to change.

## 2. Analyze events by Diagnostic Settings

Let's do a simple query first to analyze the number of events by Operation Name. 

> **Note**: Each row in AzureDiagnostic represents an event for specific Operation or category. Some SQL actions may result in generating multiple events of different types.



In [ ]:
AzureDiagnostics
| summarize count() by OperationName

The above query's equivalent in SQL is:

```
SELECT OperationName, COUNT(*) AS [count_]
FROM AzureDiagnostics
GROUP BY OperationName
```

Count my Azure SQL DB events by category / diagnostic settings.

In [ ]:
AzureDiagnostics
| where LogicalServerName_s == "trisqldemoservertaiob"
| where TimeGenerated >= ago(5d)
| summarize count() by Category


## 3. Performance troubleshooting Query (from Azure Portal)

Potentially a query or deadlock on the system that could lead to poor performance. The following is a query suggested by Azure Portal.

In [ ]:
AzureMetrics
| where ResourceProvider == "MICROSOFT.SQL"
| where TimeGenerated >=ago(1d)
| where MetricName in ('deadlock')
| parse _ResourceId with * "/microsoft.sql/servers/" Resource // subtract Resource name for _ResourceId
| summarize Deadlock_max_60Mins = max(Maximum) by Resource, MetricName

# AzureMetrics

This is a sample query to dig into AzureMetrics

In [ ]:
AzureMetrics
| project-away TenantId, ResourceId, SubscriptionId, _ResourceId, ResourceGroup // hide sensitive info
| project TimeGenerated, MetricName, Total, Count, UnitName
| take 10

# AzureDiagnostics

This is a sample query to dig into AzureDiagnostics. This table tends to have more details than AzureMetrics.

In [ ]:
AzureDiagnostics
| project-away TenantId, ResourceId, SubscriptionId, ResourceGroup, _ResourceId // Hide sensitive columns :) 
| project TimeGenerated, Category, OperationName
| take 10

## Analyze (non-audit) Events

In [ ]:
AzureDiagnostics
| summarize event_count = count() by bin(TimeGenerated, 2d), OperationName
| where OperationName <> "AuditEvent"
| evaluate pivot(OperationName, sum(event_count))
| sort by TimeGenerated asc

In [ ]:
AzureDiagnostics
| summarize event_count=count() by bin(TimeGenerated, 1d), OperationName
// | where OperationName <> "AuditEvent"

## Deadlock Analysis

In [ ]:
AzureDiagnostics
| where OperationName == "DeadlockEvent"
| project TimeGenerated, Category, Resource, OperationName, Type, deadlock_xml_s
| sort by TimeGenerated desc
| take 50

## Find the deadlock query plan

In [ ]:
AzureDiagnostics
| where OperationName == "DeadlockEvent"
| extend d = parse_xml(deadlock_xml_s)
| project TimeGenerated, QueryPlanHash = d.deadlock.["process-list"].process[0].executionStack.frame[0]["@queryplanhash"], QueryHash = d.deadlock.["process-list"].process[0].executionStack.frame[0]["@queryhash"]
| take 50

## Query Store Runtime Statistics Events

In [ ]:
AzureDiagnostics
| where OperationName == "QueryStoreRuntimeStatisticsEvent"
| project TimeGenerated, query_hash_s, statement_sql_handle_s, query_plan_hash_s
| take 10

## Analyze Errors

In [ ]:
AzureDiagnostics
| where OperationName == "ErrorEvent"
| extend ErrorNumber =  tostring(error_number_d) 
| summarize event_count=count() by EventTime = bin(TimeGenerated, 5m),  ErrorNumber
| evaluate pivot(ErrorNumber, sum(event_count))
| sort by EventTime asc


## Find Deleted table

In [ ]:
AzureDiagnostics
| where action_name_s in ('BATCH COMPLETED')
| project TimeGenerated, Category, action_name_s, statement_s
| where statement_s contains "DROP TABLE"
| sort by TimeGenerated desc 
| take 10

